In [276]:
# First lets import all the libraries we are gonna use,
import functions as fun 
import pandas as pd


In [277]:
# Now lets open and define the DataFrame,
df = pd.read_csv('/mnt/c/Users/marcn/Documents/Ironhack/sharky/Data/attacks.csv', encoding="ISO-8859-1")
new_dict = {i: i.strip().replace(" ", "_").replace('.','').lower() for i in df.columns}
df.rename(columns=new_dict, inplace=True)
df.drop(columns= ['unnamed:_22', 'unnamed:_23','area','location', 'type','injury', 'fatal_(y/n)', 'time','investigator_or_source', 'pdf', 'href_formula', 'href','original_order' ], inplace=True) # We are dropping the unnecesary data for our study
df.dropna(axis = 0, how = 'all', inplace = True)
# Let's make it a little more easy to work with by renaming columns:


In [278]:
# Once with our dataframe defined we can start cleaning the data, lets first do a little further exploration on it,
# Let's see what data contains information about the Year when the attack happened,
a = list(df.columns)
print(a)

['case_number', 'date', 'year', 'country', 'activity', 'name', 'sex', 'age', 'species', 'case_number1', 'case_number2']


In [279]:
# Let's check the nulls,
df.isnull().sum() # It is quite obvious we can drop the last 2 columns

case_number        1
date            2401
year            2403
country         2451
activity        2945
name            2611
sex             2966
age             5232
species         5239
case_number1    2401
case_number2    2401
dtype: int64

In [280]:
#some important observations:
""" We see that many cases are not dated, just labeld as before some year. We can't use this data, it is not precise.
Also there is some data that is labeled with a range between 2 years. The range is never higher than 20 years in the observed sample, so i am going
to use this data, eventhough it is not very precise it is bounded. """
df[df[['case_number','date','year']].notnull().all(1)].sample(3)


,case_number,date,year,country,activity,name,sex,age,species,case_number1,case_number2
15,2018.05.12,12-May-2018,2018.0,SOUTH AFRICA,Surfing,male,M,NaN,NaN,2018.05.12,2018.05.12
3947,1960.11.22,22-Nov-1960,1960.0,USA,"Fell overboard, prop slashed arm",Sarah Peggy Sargent,F,51,Shark involvement prior to death was not confi...,1960.11.22,1960.11.22
826,2012.01.13,13-Jan-2012,2012.0,USA,Surfing,Steve Harnack,M,53,White shark,2012.01.13,2012.01.13


In [281]:
df.loc[df['year'].isnull()].tail()


,case_number,date,year,country,activity,name,sex,age,species,case_number1,case_number2
8698,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8699,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8700,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8701,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25722,xx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Cleaning by date,
The feature I want to focus on is the year of the attack, so what I am going to do is try to assing a year to each attack, if possible. First, I observed that 125 of the attacks are assigned year '0.0', but for most of the cases in the date column there is more information about the date, all that have a date with the string 'Before' in it are not going to count for the study, and the ones with a range in them are going to be assigned the date in the middle of the range. Also, I am going to take in account all the attacks that happened A.D, and drop everything B.C. 

The order of cleaning will go as follows:
1. First i am going locate all the data that has a NaN or a 0.0 in the Year column. And work with those.
2. After that, I am going to grop all the rows with Dates that have 'B.C' in them.
3. Now, the Dates that go as: '\d{4}-\d{4}' are going to be replaced with the year in between. 
4. In some rare cases there is also a string such as: Late 1970's, we are going to try and find an s afer a number, if we find it we are going to return number+5 (in this cas 1975, so it is more or less precise with both earlies and late).
5. And finally, for the rest with a NaN or a 0.0 in their year column, I am going to try and find a string with '\d{4}|\d{3}' in them, and replace the year value for the found value. 



In [282]:
df.case_number.sample(10)

4970    1933.02.15.R
6484               0
489       2014.10.11
6242         ND.0064
2924    1983.08.13.b
4821      1938.03.08
347     2015.10.09.b
3156      1977.02.04
8638               0
6046    1847.00.00.b
Name: case_number, dtype: object

In [283]:
df['year'] = df.year.apply(fun.clean_column)
df['case_number'] = df.case_number.apply(fun.clean_column)
df['case_number1'] = df.case_number1.apply(fun.clean_column)
df['case_number2'] = df.case_number2.apply(fun.clean_column)
#df['date'] = df.date.apply(fun.clean_column)
df.dropna(axis = 0, how = 'all', inplace = True)

In [284]:
df.case_number.sample(10)

3722    1963.0
985     2010.0
3378    1970.0
2500    1993.0
6094    1829.0
493     2014.0
5790    1883.0
991     2010.0
2093    1999.0
945     2011.0
Name: case_number, dtype: float64

In [285]:
df['year'] = fun.year_replacement([df['year'], df.date, df.case_number, df.case_number1, df.case_number2])

AttributeError: module 'functions' has no attribute 'year_replacement'

In [ ]:
# Let's now clean the country column and make it more easy to read:
df['country'] = df.country.apply(lambda x: str(x).strip().replace(" ", "_").lower())

In [286]:
# let's clean the data that yields information about the people that were attacked by sharks,
df['name'] = df['name'].apply(fun.clean_names)

AttributeError: module 'functions' has no attribute 'clean_names'

In [287]:
df['sex'] = df.sex.apply(fun.clean_sex)

AttributeError: module 'functions' has no attribute 'clean_sex'